# Setup

To run the example, a file named `secret.py` must be created in the notebooks
folder with the following content:

```python
# Configuration settings

# SLIPO workbench installation
BASE_URL = 'http://192.168.1.8:8080'

# SLIPO API key
API_KEY = ''
```

The file must be imported before creating a new context:

```python
from secret import BASE_URL, API_KEY
```

In [ ]:
# Add slipoframes to path
import sys
sys.path.append('..')

In [ ]:
# Create new context
from slipoframes.context import SlipoContext

from secret import BASE_URL, API_KEY

ctx = SlipoContext(
    base_url = BASE_URL,
    requires_ssl = False,
    api_key = API_KEY
)

# Transform operation

Next we are going to:

* Upload a file to the local file system
* Execute a transform operation
* Check the operation status
* Download the output RDF dataset localy
* Perform a simple query on the RDF graph using  [gastrodon](https://github.com/paulhoule/gastrodon). 

In [ ]:
# Upload file
ctx.file_upload('../datasets/osmpois-berlin.csv', 'notebook/osmpois-berlin.csv', overwrite=True)

In [ ]:
# Browse remote user file system
df_files = ctx.file_browse(sort_col='size', format_size=True, sort_asc=False)

df_files[df_files['name'].str.startswith("osmpois")]

In [ ]:
# Convert CSV file to N-Triples
ctx.transform_csv(
    'notebook/osmpois-berlin.csv',
    'OSM_Europe',
    attrCategory='kwds',
    attrKey='id',
    attrName='name',
    attrX='lon',
    attrY='lat',
    defaultLang='en',
    delimiter=';',
    encoding='UTF-8',
    featureSource='OSM',
    quote='',
    sourceCRS='EPSG:4326',
    targetCRS='EPSG:4326'
)

In [ ]:
# Check process status. Use process id and version values from previous step
df_files = ctx.process_status(274, 1)
df_files

In [ ]:
# Download output file. Use process id, version and file id from previous steps
ctx.process_file_download(274, 1, 5903, '../datasets/osmpois-berlin.nt')

In [ ]:
%%capture
%pip install gastrodon

In [ ]:
# Create graph
from rdflib import Graph,URIRef
from gastrodon import LocalEndpoint,one,QName

g = Graph()
g.parse('../datasets/osmpois-berlin.nt', format="nt")

In [ ]:
# Get graph size
len(g)

In [ ]:
# Create an endpoint and execute a simple query
e=LocalEndpoint(g)

properties = e.select("""
   SELECT ?p (COUNT(*) AS ?cnt) {
      ?s ?p ?o .
   } GROUP BY ?p ORDER BY DESC(?cnt)
""")

properties